In [20]:
import pandas as pd
import yaml
import logging
from datetime import datetime
from sqlalchemy import create_engine, Table, Column, Integer, String, Float, MetaData

def configure_logging(log_file_path):
    """Configures logging to track the ETL process."""
    logging.basicConfig(
        filename=log_file_path,
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s'
    )

def load_config(file_path):
    """Loads configuration from a YAML file."""
    with open(file_path, 'r') as config_file:
        return yaml.safe_load(config_file)

def extract_data(file_path):
    """Extracts relevant columns from the Excel file."""
    selected_columns = [
        'Retailer ID', 'Retailer', 'Region', 'State', 'City', 'Product',
        'Price per Unit', 'Units Sold', 'Total Sales', 'Sales Method', 'Invoice Date'
    ]
    data = pd.read_excel(file_path, header=4, usecols=selected_columns)
    logging.info("Data extraction completed.")
    return data

def transform_data(data):
    """Transforms data: rounds numeric columns, converts dates, and extracts year."""
    columns_to_round = ['Price per Unit', 'Total Sales']
    data[columns_to_round] = data[columns_to_round].round(0).astype(int)

    if 'Invoice Date' in data.columns:
        data['Invoice Date'] = pd.to_datetime(data['Invoice Date'], errors='coerce')
        data['Year'] = data['Invoice Date'].dt.year

    logging.info("Data transformation completed.")
    return data

def write_to_xlsx(file_path, data):
    """Saves transformed data to an Excel file."""
    data.to_excel(file_path, index=False)
    logging.info(f"Transformed data saved to {file_path}.")

def create_database_schema(engine):
    """Creates a database schema with the required structure."""
    metadata = MetaData()
    sales_data = Table(
        'Sales_data', metadata,
        Column('Retailer ID', Integer, primary_key=True),
        Column('Retailer', String),
        Column('Region', String),
        Column('State', String),
        Column('City', String),
        Column('Product', String),
        Column('Price per Unit', Float),
        Column('Units Sold', Integer),
        Column('Total Sales', Float),
        Column('Sales Method', String),
        Column('Invoice Date', String)  # Ensure proper format for date storage
    )
    metadata.create_all(engine)
    logging.info("Database schema created.")

def load_data_to_database(engine, table_name, data):
    """Loads transformed data into the database."""
    data.to_sql(table_name, engine, if_exists='replace', index=False)
    logging.info("Data successfully loaded into the database.")

if __name__ == "__main__":
    # Load configuration
    config = load_config('config.yaml')
    excel_file_path = config['excel_file_path']
    log_file_path = config['log_file_path']
    db_config = config['db_config']

    # Configure logging
    configure_logging(log_file_path)
    logging.info("ETL process started.")

    # Extract data
    data = extract_data(excel_file_path)

    # Transform data
    transformed_data = transform_data(data)

    # Save transformed data to Excel (optional step)
    write_to_xlsx('transformed_data.xlsx', transformed_data)

    # Create database connection
    engine = create_engine(
        f'postgresql://{db_config["user"]}:{db_config["password"]}@'
        f'{db_config["host"]}:{db_config["port"]}/{db_config["database"]}'
    )

    # Create database schema (if needed)
    create_database_schema(engine)

    # Load data into the database
    load_data_to_database(engine, 'Sales_data', transformed_data)

    logging.info("ETL process completed successfully.")